In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [2]:
tickers = ['GOOG', 'AMZN', 'TSLA', 'MSFT', 'AAPL']
stock_data = yf.download(tickers, start="2022-01-01", end="2024-02-23")['Adj Close']

[*********************100%%**********************]  5 of 5 completed


In [3]:
returns_data = stock_data.pct_change().dropna()

We have these 5 stocks in our portfolio, we will now calculate the Value at Risk (VaR) and expected shortfall (ES) of this portfolio

In [14]:
#Assigning weights
random_numbers = np.random.rand(5)
normalized_numbers = random_numbers / random_numbers.sum()
normalized_numbers

array([0.28356852, 0.22382323, 0.21848678, 0.07992626, 0.19419521])

VaR using parametric approach is VaR = Mean + z*stdev

(Assuming normal data so mean = 0)

Portfolio variance = ∑∑ Wi * Wj * Covariance(i,j)

In [15]:
#Using the historical simulation approach
def VaR(df,confidence,days):
    variance = 0
    z_score = norm.ppf(confidence)
    for i in range(5):
      for j in range(5):
        variance = variance + normalized_numbers[i] * normalized_numbers[j] * np.cov(df.iloc[:,i],df.iloc[:,j])[0][1]
    stdev = np.sqrt(variance)
    #Calculating the 10 day VaR at 95% confidence interval
    var = stdev * z_score *np.sqrt(days)
    return var

print(f"10 day Value at Risk (VaR) for the portfolio at 95% confidence level : {VaR(returns_data,0.95,10)*100}%" )

10 day Value at Risk (VaR) for the portfolio at 95% confidence level : 10.488219916081519%


For calculating the expected shortfall, we will consider intervals from 95% and all the way upto 99.99% to get the most accurate ES

In [18]:
#Conditional Value at Risk (cVaR) OR Expected shortfall(ES)
def ES(start,end):
  es = 0.0
  i = start
  count=0
  while i <= end:
    es+= VaR(returns_data,i,10)
    i += 0.0001
    count+=1
  return es*100/count

In [19]:
ES(0.95,0.9999)

13.136632138747913